In [0]:
import numpy as np

In [2]:
!pip install ortools

     |████████████████████████████████| 27.9MB 143kB/s 
     |████████████████████████████████| 1.3MB 42.4MB/s 
  Found existing installation: protobuf 3.10.0
    Uninstalling protobuf-3.10.0:
      Successfully uninstalled protobuf-3.10.0


# **Lendo arquivo**

In [3]:
path = 'Problema.txt'
arquivo = open(path,'r')
numeros = []

for linha  in arquivo:
  linha = linha.strip()
  numeros.append(linha)

arquivo.close()
numeros

['3 2', '5 10 8', '3 5 2 60', '4 4 4 72']

In [0]:
x1 = numeros[0].split(' ') #variaveis e restricoes:
c = numeros[1].split(' ') #Coecicientes das variaveis na funcao objetivo

In [0]:
rest = len(numeros)-2 #Numeros de restrições
var = int(x1[0]) #Números de variáveis

# ***Formatação para entrada do modelo***
  O solver exige que os arrays sejam da seguinte forma:


*   Os arrays das restrições precisam ser separados da igualdade
  
  Ex: 
  
        x1 + x2 <= 10 e x1 + x2 <= 20
  é necessário um array [ 10, 20 ] e outro com os coeficientes das restrições

In [6]:
a = [0]*rest #inicializando array com o numero de restricoes
b = [0]*rest 

for i in range(2, len(numeros)): #começando em dois pois oq vem depois da linha 2 sao as restricoes
    aa = numeros[i].split(' ')
    b[i-2] = aa[len(aa)-1]
    del(aa[len(aa)-1])
    a[i-2] = aa
print(a, b, c)

[['3', '5', '2'], ['4', '4', '4']] ['60', '72'] ['5', '10', '8']


# ***Convertendo os arrays em double***
  A biblioteca or-tolls exige que as variáveis sejam desta forma

In [7]:
a = np.double( a )
b = np.double( b )
c = np.double( c )

print(a, b, c)

[[3. 5. 2.]
 [4. 4. 4.]] [60. 72.] [ 5. 10.  8.]


# **Definindo tipos**

*  Para resolver o problema em inteiro ou não
  
        intOrNot = True
        intOrNot = False

*  Setar igualdades das restrições
  
        LessMoreOrEqual = 'MoreOrEqual'
        LessMoreOrEqual = 'LessOrEqual'
      
* Setar função objetivo

        MaxOrMin = 'Min'
        MaxOrMin = 'Max'


In [0]:
IntOrNot = False
LessMoreOrEqual = 'MoreOrEqual'
MaxOrMin = 'Min'

# ***Criando o modelo***

In [0]:
def create_data_model(A, B, C, num_vars, num_rest):
  data = {}
  data['constraint_coeffs'] = A
  data['bounds'] = B
  data['obj_coeffs'] = C
  data['num_vars'] = num_vars
  data['num_constraints'] = num_rest
  return data

In [0]:
data = create_data_model(a, b, c, var, rest)

# **Main**

In [0]:
from ortools.linear_solver import pywraplp

def main(IntOrNot, LessMoreOrEqual, MaxOrMin):

  solver = pywraplp.Solver('simple_mip_program', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

  infinity = solver.infinity()
  x = {}

  if IntOrNot == False:
    for j in range(data['num_vars']):
        x[j] = solver.NumVar(0, infinity, 'x[%i]' % j) #Variáveis positivas
  else:
    for j in range(data['num_vars']):
        x[j] = solver.IntVar(0, infinity, 'x[%i]' % j) #Variáveis positivas inteiras
  
  print('Numero de variaveis =', solver.NumVariables())
  
  if LessMoreOrEqual == 'LessOrEqual':
    for i in range(data['num_constraints']):
        constraint = solver.RowConstraint(0, data['bounds'][i], '')#limite inferior, superor e nome da restrição
        for j in range(data['num_vars']):
            constraint.SetCoefficient(x[j], data['constraint_coeffs'][i][j]) 
  
  if LessMoreOrEqual == 'MoreOrEqual':
    for i in range(data['num_constraints']):
      constraint = solver.RowConstraint(data['bounds'][i], infinity, '')#limite inferior, superior e nome da restrição
      for j in range(data['num_vars']):
        constraint.SetCoefficient(x[j], data['constraint_coeffs'][i][j]) 
  
  print('Numero de restriçoes =', solver.NumConstraints())

  objective = solver.Objective()
  
  for j in range(data['num_vars']):
      objective.SetCoefficient(x[j], data['obj_coeffs'][j])
  
  if MaxOrMin == 'Max':
    objective.SetMaximization()
  else:
    objective.SetMinimization()

  status = solver.Solve()

  solution_value = []
  if status == pywraplp.Solver.OPTIMAL:
      print('Valor ótimo = ', solver.Objective().Value())
      for j in range(data['num_vars']):
          print(x[j].name(), ' = ', x[j].solution_value())
          solution_value.append(x[j].solution_value()) 
      print()
      print('Problema resolvido em %f milliseconds' % solver.wall_time())
      print('Problema resolvido em %d iteracoes' % (solver.iterations()+1))
      return solution_value #retornando os valores soluções
  else:
      print('Nao tem solucao otima.')

In [21]:
solution_primal = main(IntOrNot, LessMoreOrEqual, MaxOrMin)

Numero de variaveis = 3
Numero de restriçoes = 2
Valor ótimo =  100.0
x[0]  =  20.0
x[1]  =  0.0
x[2]  =  0.0

Problema resolvido em 8.000000 milliseconds
Problema resolvido em 1 iteracoes


# **Dual**

In [0]:
aT = np.transpose( a )
bT = c
cT = b

num_varsT = rest
num_restT = var

IntOrNot = False
LessMoreOrEqual = 'LessOrEqual'
MaxOrMin = 'Max'

In [0]:
data = create_data_model(aT, bT, cT, num_varsT, num_restT)

In [24]:
print("========== DUAL ==========")
solution_dual = main(IntOrNot, LessMoreOrEqual, MaxOrMin)

========== DUAL ==========
Numero de variaveis = 2
Numero de restriçoes = 3
Valor ótimo =  99.99999999999999
x[0]  =  1.6666666666666665
x[1]  =  0.0

Problema resolvido em 6.000000 milliseconds
Problema resolvido em 1 iteracoes


# **Variaveis de folga e excessos**

In [0]:
def FolgasOrExces(solution, a, b):
  folgasOuExcesso=[]
  
  print("Restrições: ", a)
  print("Lado direito da igualdade: ", b)
  print("Solução: ", solution)

  soma = 0
  for i in range(0, len(a)): #somatório de cada restrição
    for j in range(0, len(a[i])): #for percorrendo cada elemento da restrição
      soma = soma + a[i][j]*solution[j] #Coeficiente da restrição vezes o valor da solução

    soma = abs(abs(soma) - abs(b[i])) #Pegando a diferença (folgas/excessos)
    folgasOuExcesso.append(soma)
    soma = 0
  
  return folgasOuExcesso

In [107]:
folgas_ou_excesso_primal = FolgasOrExces(solution_primal, a, b)
print()
print("Valores de folgas/excesso: ", folgas_ou_excesso_primal)

Restrições:  [[3. 5. 2.]
 [4. 4. 4.]]
Lado direito da igualdade:  [60. 72.]
Solução:  [20.0, 0.0, 0.0]

Valores de folgas/excesso:  [0.0, 8.0]


In [105]:
folgas_ou_excesso_dual = FolgasOrExces(solution_dual, aT, bT)
print()
print("Valores de folgas/excesso: ", folgas_ou_excesso_dual)

Restrições:  [[3. 4.]
 [5. 4.]
 [2. 4.]]
Lado direito da igualdade:  [ 5. 10.  8.]
Solução:  [1.6666666666666665, 0.0]

Valores de folgas/excesso:  [0.0, 1.6666666666666679, 4.666666666666667]


# **Folgas/Excessos complementares**

In [0]:
def complementares (solution, folgasOrExcess):
  complemento = []

  for i in range (len(solution)):
    complemento.append(solution[i]*folgasOrExcess[i]) #Multiplicação do valor da solulçao vezes a folga ou excesso do complementar
  
  return complemento

In [102]:
complementar_primal = complementares(solution_primal, folgas_ou_excesso_dual)
print("Valor da solução vezes o complementar excesso/folga: ", complementar_primal)

Valor da solução vezes o complementar excesso/folga:  [0.0, 0.0, 0.0]


In [103]:
complementar_dual = complementares(solution_primal, folgas_ou_excesso_dual)
print("Valor da solução do dual vezes o complementar excesso/folga: ", complementar_dual)

Valor da solução do dual vezes o complementar excesso/folga:  [0.0, 0.0, 0.0]
